# Ring Resonator Filter Simulation

This notebook demonstrates a complete workflow for simulating a ring resonator optical filter using the beamz FDTD library:

1. **Design Creation** - Build ring resonator geometry using gdsfactory
2. **2D FDTD Simulation** - Run wavelength sweep to extract transmission spectrum
3. **Spectral Analysis** - Calculate Q-factor, FSR, and resonance wavelengths
4. **3D Verification** - Quick 3D simulation to verify mode propagation

Ring resonators are fundamental building blocks in photonic integrated circuits, used for filtering, wavelength-selective routing, and sensing applications. They work by trapping light at specific resonant wavelengths through constructive interference.

---

In [ ]:
from beamz import *
from beamz.design.io import import_gds
import numpy as np
import matplotlib.pyplot as plt
import gdsfactory as gf
from scipy.signal import find_peaks
from scipy.optimize import curve_fit

## 1. Design Parameters

We define the physical parameters for our ring resonator. The design uses silicon nitride (Si3N4) waveguides on silicon dioxide (SiO2) cladding - a common platform for visible and near-IR photonics.

In [ ]:
# Material refractive indices
N_CORE = 2.04    # Si3N4 core
N_CLAD = 1.444   # SiO2 cladding

# Ring resonator geometry
RING_RADIUS = 6 * um       # Ring radius
WG_WIDTH = 0.5 * um        # Waveguide width
GAP = 0.5 * um             # Coupling gap between bus and ring
BUS_LENGTH = 20 * um       # Length of bus waveguide

# Simulation wavelength range
CENTER_WL = 1.55 * um      # Center wavelength (telecom C-band)
WL_SPAN = 0.05 * um        # Wavelength sweep span
NUM_WAVELENGTHS = 51       # Number of wavelengths (increased for better resolution)

# PML boundary thickness
PML_THICKNESS = 1.5 * CENTER_WL

print(f"Ring radius: {RING_RADIUS*1e6:.1f} um")
print(f"Waveguide width: {WG_WIDTH*1e9:.0f} nm")
print(f"Coupling gap: {GAP*1e9:.0f} nm")
print(f"Wavelength range: {(CENTER_WL-WL_SPAN/2)*1e9:.1f} - {(CENTER_WL+WL_SPAN/2)*1e9:.1f} nm")
print(f"Number of wavelengths: {NUM_WAVELENGTHS}")

## 2. Create Ring Resonator with gdsfactory

We use gdsfactory to create the ring resonator geometry. This represents a typical photonic IC design workflow where layouts are created in gdsfactory and then simulated with an FDTD solver.

In [ ]:
@gf.cell
def ring_resonator_filter(
    radius: float = 6.0,           # Ring radius in microns
    width: float = 0.5,            # Waveguide width in microns
    gap: float = 0.2,              # Coupling gap in microns
    bus_length: float = 20.0,      # Bus waveguide length in microns
    layer: tuple = (1, 0)          # GDS layer
) -> gf.Component:
    """Create a ring resonator with input and output bus waveguides."""
    c = gf.Component()
    
    # Create the ring
    ring = c << gf.components.ring(
        radius=radius,
        width=width,
        layer=layer
    )
    
    # Create the input bus waveguide (bottom)
    bus_input = c << gf.components.straight(
        length=bus_length,
        width=width
    )
    
    # Create the output bus waveguide (top)
    bus_output = c << gf.components.straight(
        length=bus_length,
        width=width
    )
    
    # Position: ring centered at origin
    # Input bus center is at y = -(radius + gap + width/2) (below ring)
    bus_input_y = -(radius + gap + width/2)
    bus_input.move((-bus_length/2, bus_input_y))
    
    # Output bus center is at y = radius + gap + width/2 (above ring)
    bus_output_y = radius + gap + width/2
    bus_output.move((-bus_length/2, bus_output_y))
    
    return c

# Create the component
ring_component = ring_resonator_filter(
    radius=RING_RADIUS * 1e6,  # Convert to microns for gdsfactory
    width=WG_WIDTH * 1e6,
    gap=GAP * 1e6,
    bus_length=BUS_LENGTH * 1e6
)

# Display the component
ring_component.plot()

In [ ]:
# Export to GDS file
gds_path = "ring_resonator_temp.gds"
ring_component.write_gds(gds_path)
print(f"GDS file saved to: {gds_path}")

## 3. Import GDS into beamz

Now we import the GDS file into beamz and set up the simulation domain with proper materials.

In [ ]:
# Import the GDS file
gds_design = import_gds(gds_path, default_depth=0)  # 2D simulation

# Calculate bounding box from imported geometry
all_vertices = []
for layer_num, structures in gds_design.layers.items():
    for structure in structures:
        all_vertices.extend(structure.vertices)

x_coords = [v[0] for v in all_vertices]
y_coords = [v[1] for v in all_vertices]

x_min, x_max = min(x_coords), max(x_coords)
y_min, y_max = min(y_coords), max(y_coords)

# Add margin for PML - CRITICAL: waveguides must extend INTO PML for proper absorption
margin_x = PML_THICKNESS  # Horizontal: waveguides extend into PML (prevents reflections)
margin_y = PML_THICKNESS + 1.5 * um  # Vertical: PML + extra padding

domain_width = (x_max - x_min) + 2 * margin_x
domain_height = (y_max - y_min) + 2 * margin_y

# Offset to center geometry in domain
x_offset = -x_min + margin_x
y_offset = -y_min + margin_y

print(f"Geometry bounds: x=[{x_min*1e6:.2f}, {x_max*1e6:.2f}] um, y=[{y_min*1e6:.2f}, {y_max*1e6:.2f}] um")
print(f"PML thickness: {PML_THICKNESS*1e6:.2f} um")
print(f"Domain size: {domain_width*1e6:.1f} x {domain_height*1e6:.1f} um")
print(f"Waveguides extend into PML: Yes (margin_x = {margin_x*1e6:.2f} um)")

In [ ]:
# Create the beamz Design with SiO2 background
design = Design(width=domain_width, height=domain_height, material=Material(N_CLAD**2))

# Add structures from GDS with Si3N4 material
# Also track the actual positions of bus waveguides after shifting
bus_y_positions = []

for layer_num, structures in gds_design.layers.items():
    for structure in structures:
        # Shift vertices to center in domain
        shifted_vertices = [(v[0] + x_offset, v[1] + y_offset) for v in structure.vertices]
        design += Polygon(vertices=shifted_vertices, material=Material(N_CORE**2))
        
        # Identify bus waveguides by their y-extent (thin horizontal structures)
        y_coords = [v[1] + y_offset for v in structure.vertices]
        y_min_struct, y_max_struct = min(y_coords), max(y_coords)
        y_extent = y_max_struct - y_min_struct
        
        # Bus waveguides have small y-extent (approximately WG_WIDTH)
        if y_extent < 2 * WG_WIDTH:
            bus_center_y = (y_min_struct + y_max_struct) / 2
            bus_y_positions.append(bus_center_y)

# Sort to get input (lower) and output (upper) bus positions
bus_y_positions.sort()
if len(bus_y_positions) >= 2:
    BUS_INPUT_Y = bus_y_positions[0]   # Bottom bus (input)
    BUS_OUTPUT_Y = bus_y_positions[-1]  # Top bus (output)
else:
    # Fallback to calculation if detection fails
    BUS_INPUT_Y = y_offset + (-(RING_RADIUS + GAP + WG_WIDTH/2))
    BUS_OUTPUT_Y = y_offset + (RING_RADIUS + GAP + WG_WIDTH/2)

# Also get x-extent of waveguides for source/monitor placement
all_x = []
for layer_num, structures in gds_design.layers.items():
    for structure in structures:
        all_x.extend([v[0] + x_offset for v in structure.vertices])
WG_X_MIN = min(all_x)
WG_X_MAX = max(all_x)

print(f"Input bus waveguide y-position: {BUS_INPUT_Y*1e6:.2f} um")
print(f"Output bus waveguide y-position: {BUS_OUTPUT_Y*1e6:.2f} um")
print(f"Waveguide x-extent: [{WG_X_MIN*1e6:.2f}, {WG_X_MAX*1e6:.2f}] um")

# Visualize the design
design.show()

## 4. Single Wavelength Test Simulation

Before running the full wavelength sweep, we run a single simulation to visualize the ring resonator behavior and verify our setup.

In [ ]:
# Calculate optimal grid parameters
WL_TEST = CENTER_WL
DX, DT = calc_optimal_fdtd_params(
    WL_TEST, N_CORE, 
    dims=2, 
    safety_factor=0.999, 
    points_per_wavelength=9
)

# Simulation time: need enough for field to build up in ring
# Ring round-trip time ~ 2*pi*R*n_eff/c
n_eff = (N_CORE + N_CLAD) / 2  # Approximate effective index
round_trip_time = 2 * np.pi * RING_RADIUS * n_eff / LIGHT_SPEED
TIME = 60 * round_trip_time  # Multiple round trips for resonance buildup

time_steps = np.arange(0, TIME, DT)

print(f"Grid resolution: {DX*1e9:.1f} nm")
print(f"Time step: {DT*1e18:.3f} as")
print(f"Number of time steps: {len(time_steps)}")
print(f"Simulation time: {TIME*1e12:.2f} ps")
print(f"Ring round-trip time: {round_trip_time*1e15:.2f} fs")

In [ ]:
# Rasterize the design
grid = design.rasterize(resolution=DX)

# Source position: on input bus waveguide, using detected positions
# Place source 2 wavelengths inside the left edge of waveguides
source_x = WG_X_MIN + 2*WL_TEST
source_y = BUS_INPUT_Y

# Monitor x-position: 2 wavelengths inside the right edge of waveguides
monitor_x_right = WG_X_MAX - 2*WL_TEST

# Create source signal
signal = ramped_cosine(
    time_steps,
    amplitude=1.0,
    frequency=LIGHT_SPEED/WL_TEST,
    ramp_duration=WL_TEST * 8 / LIGHT_SPEED,
    t_max=TIME / 3
)

# Mode source for waveguide excitation
source = ModeSource(
    grid=grid,
    center=(source_x, source_y),
    width=WG_WIDTH * 3.5,
    wavelength=WL_TEST,
    pol="tm",
    signal=signal,
    direction="+x"
)

# Input monitor (for normalization) - on input bus, after source
monitor_input = Monitor(
    start=(source_x + 2*WL_TEST, source_y - WG_WIDTH),
    end=(source_x + 2*WL_TEST, source_y + WG_WIDTH),
    name="input"
)

# Through port monitor (output of input bus waveguide)
monitor_through = Monitor(
    start=(monitor_x_right, source_y - WG_WIDTH),
    end=(monitor_x_right, source_y + WG_WIDTH),
    name="through"
)

# Output port monitor (on output bus waveguide at top)
monitor_output = Monitor(
    start=(monitor_x_right, BUS_OUTPUT_Y - WG_WIDTH),
    end=(monitor_x_right, BUS_OUTPUT_Y + WG_WIDTH),
    name="output"
)

print(f"Waveguide x-extent: [{WG_X_MIN*1e6:.2f}, {WG_X_MAX*1e6:.2f}] um")
print(f"Source position: ({source_x*1e6:.2f}, {source_y*1e6:.2f}) um")
print(f"Through monitor: x={monitor_x_right*1e6:.2f} um, y={source_y*1e6:.2f} um")
print(f"Output monitor: x={monitor_x_right*1e6:.2f} um, y={BUS_OUTPUT_Y*1e6:.2f} um")

In [ ]:
# Create and run simulation with animation
sim = Simulation(
    design=design,
    devices=[source, monitor_input, monitor_through, monitor_output],
    boundaries=[PML(edges='all', thickness=PML_THICKNESS)],
    time=time_steps,
    resolution=DX
)

# Run with live animation to see the coupling
sim.run(
    animate_live="Ez",
    animation_interval=20,
    cmap="twilight_zero",
    clean_visualization=True,
    #axis_scale=[-1e-4, 1e-4],
    jupyter_live=True,
    store_animation=True
)

# Live animation works automatically!
#result = sim.run(animate_live="Ez")

# After simulation, replay:
#result['animation'].get_animation()  # HTML5 video player
#result['animation'].get_widget()     # Slider + play button

# Script mode (unchanged)
#sim.run(animate_live="Ez")  # Works exactly as before

# Explicit control
#sim.run(animate_live="Ez", jupyter_live=False)  # Force script mode
#sim.run(animate_live="Ez", store_animation=False)  # Don't store frames

## 5. Wavelength Sweep for Transmission Spectrum

To extract the transmission spectrum and identify resonances, we run simulations at multiple wavelengths and measure the transmitted power at the through port.

In [ ]:
def run_wavelength_simulation(wavelength, design, grid, dx, dt, domain_width, bus_input_y, bus_output_y, 
                               wg_x_min, wg_x_max, pml_thickness, 
                               n_core, n_clad, wg_width, ring_radius, verbose=False):
    """Run a single wavelength simulation and return transmission data.
    
    Args:
        grid: Pre-rasterized grid (reused across wavelengths - no rerasterization!)
        dx: Fixed grid resolution
        dt: Fixed time step
    """
    
    # Time parameters - use fixed dx/dt, don't recalculate
    n_eff = (n_core + n_clad) / 2
    round_trip_time = 2 * np.pi * ring_radius * n_eff / LIGHT_SPEED
    sim_time = 60 * round_trip_time
    time_steps = np.arange(0, sim_time, dt)
    
    # Source position: on input bus, using detected waveguide x-extent
    source_x = wg_x_min + 2*wavelength
    monitor_x_right = wg_x_max - 2*wavelength
    
    signal = ramped_cosine(
        time_steps, 
        amplitude=1.0, 
        frequency=LIGHT_SPEED/wavelength,
        ramp_duration=wavelength * 8 / LIGHT_SPEED, 
        t_max=sim_time / 3
    )
    
    source = ModeSource(
        grid=grid,  # Reuse pre-rasterized grid
        center=(source_x, bus_input_y),
        width=wg_width * 3.5,
        wavelength=wavelength,
        pol="tm",
        signal=signal,
        direction="+x"
    )
    
    # Monitors using detected waveguide positions
    monitor_input = Monitor(
        start=(source_x + 2*wavelength, bus_input_y - wg_width),
        end=(source_x + 2*wavelength, bus_input_y + wg_width),
        name="input"
    )
    
    monitor_through = Monitor(
        start=(monitor_x_right, bus_input_y - wg_width),
        end=(monitor_x_right, bus_input_y + wg_width),
        name="through"
    )
    
    monitor_output = Monitor(
        start=(monitor_x_right, bus_output_y - wg_width),
        end=(monitor_x_right, bus_output_y + wg_width),
        name="output"
    )
    
    # Run simulation (no animation for sweep)
    sim = Simulation(
        design=design,
        devices=[source, monitor_input, monitor_through, monitor_output],
        boundaries=[PML(edges='all', thickness=pml_thickness)],
        time=time_steps,
        resolution=dx  # Use fixed resolution
    )
    sim.run()
    
    # Extract power data (use last 25% for steady state)
    n_steady = len(monitor_input.power_history) // 4
    input_power = np.mean(monitor_input.power_history[-n_steady:])
    through_power = np.mean(monitor_through.power_history[-n_steady:])
    output_power = np.mean(monitor_output.power_history[-n_steady:])
    
    # Transmission (through port) and drop (output port)
    transmission = through_power / input_power if input_power > 0 else 0
    drop = output_power / input_power if input_power > 0 else 0
    
    if verbose:
        print(f"  WL={wavelength*1e9:.1f}nm: T={transmission:.4f}, Drop={drop:.4f}")
    
    return {
        'wavelength': wavelength,
        'transmission': transmission,
        'drop': drop,
        'input_power': input_power,
        'through_power': through_power,
        'output_power': output_power
    }

In [ ]:
# Define wavelength range
wavelengths = np.linspace(CENTER_WL - WL_SPAN/2, CENTER_WL + WL_SPAN/2, NUM_WAVELENGTHS)

# Pre-compute grid ONCE using shortest wavelength for best accuracy
shortest_wl = wavelengths.min()
dx_sweep, dt_sweep = calc_optimal_fdtd_params(
    shortest_wl, N_CORE, 
    dims=2, 
    safety_factor=0.999, 
    points_per_wavelength=8
)

# Rasterize ONCE - reuse for all wavelengths (no rerasterization!)
print("Pre-rasterizing grid (one-time operation)...")
grid_sweep = design.rasterize(resolution=dx_sweep)
print(f"Grid resolution: {dx_sweep*1e9:.1f} nm (based on shortest wavelength)")

# Run sweep
print(f"\nRunning wavelength sweep...")
print(f"Wavelength range: {wavelengths[0]*1e9:.1f} - {wavelengths[-1]*1e9:.1f} nm")
print(f"Number of simulations: {len(wavelengths)}")
print()

results = []
for i, wl in enumerate(wavelengths):
    print(f"[{i+1}/{len(wavelengths)}] Wavelength: {wl*1e9:.1f} nm")
    result = run_wavelength_simulation(
        wavelength=wl,
        design=design,
        grid=grid_sweep,      # Pass pre-rasterized grid
        dx=dx_sweep,          # Fixed resolution
        dt=dt_sweep,          # Fixed time step
        domain_width=domain_width,
        bus_input_y=BUS_INPUT_Y,
        bus_output_y=BUS_OUTPUT_Y,
        wg_x_min=WG_X_MIN,
        wg_x_max=WG_X_MAX,
        pml_thickness=PML_THICKNESS,
        n_core=N_CORE,
        n_clad=N_CLAD,
        wg_width=WG_WIDTH,
        ring_radius=RING_RADIUS,
        verbose=True
    )
    results.append(result)

# Extract arrays
wl_array = np.array([r['wavelength'] for r in results])
transmission_array = np.array([r['transmission'] for r in results])
drop_array = np.array([r['drop'] for r in results])

print("\nWavelength sweep complete!")

## 6. Plot Transmission Spectrum

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
axes[0].plot(wl_array * 1e9, transmission_array, 'b-o', linewidth=2, markersize=6)
axes[0].set_xlabel('Wavelength (nm)', fontsize=12)
axes[0].set_ylabel('Transmission', fontsize=12)
axes[0].set_title('Ring Resonator Transmission Spectrum', fontsize=14)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(0, 1.1)

# dB scale
transmission_db = 10 * np.log10(np.maximum(transmission_array, 1e-10))
axes[1].plot(wl_array * 1e9, transmission_db, 'r-o', linewidth=2, markersize=6)
axes[1].set_xlabel('Wavelength (nm)', fontsize=12)
axes[1].set_ylabel('Transmission (dB)', fontsize=12)
axes[1].set_title('Transmission Spectrum (dB scale)', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Resonance Analysis

We extract key ring resonator parameters:
- **Resonance wavelengths**: Dips in the transmission spectrum
- **Free Spectral Range (FSR)**: Spacing between adjacent resonances
- **Quality factor (Q)**: Measure of resonance sharpness (Q = λ/Δλ)

In [ ]:
def lorentzian(x, x0, gamma, amplitude, offset):
    """Lorentzian lineshape for fitting resonance dips."""
    return offset - amplitude * (gamma/2)**2 / ((x - x0)**2 + (gamma/2)**2)

def find_resonances(wavelengths, transmission, min_depth=0.05):
    """Find resonance dips in transmission spectrum."""
    # Invert transmission to find peaks (which correspond to dips)
    inverted = 1 - transmission
    
    # Find peaks in inverted spectrum
    peaks, properties = find_peaks(inverted, height=min_depth, distance=2)
    
    resonance_wls = wavelengths[peaks]
    resonance_depths = inverted[peaks]
    
    return resonance_wls, resonance_depths, peaks

def calculate_q_factor(wavelengths, transmission, resonance_idx, fit_width=3):
    """Calculate Q-factor by fitting Lorentzian to resonance."""
    # Extract region around resonance
    start_idx = max(0, resonance_idx - fit_width)
    end_idx = min(len(wavelengths), resonance_idx + fit_width + 1)
    
    wl_fit = wavelengths[start_idx:end_idx]
    t_fit = transmission[start_idx:end_idx]
    
    if len(wl_fit) < 4:
        return None, None
    
    # Initial guess
    x0_guess = wavelengths[resonance_idx]
    gamma_guess = (wavelengths[-1] - wavelengths[0]) / 20
    amplitude_guess = 1 - transmission[resonance_idx]
    offset_guess = 1.0
    
    try:
        popt, _ = curve_fit(
            lorentzian, wl_fit, t_fit,
            p0=[x0_guess, gamma_guess, amplitude_guess, offset_guess],
            bounds=([wl_fit[0], 1e-12, 0, 0], [wl_fit[-1], wl_fit[-1]-wl_fit[0], 2, 2]),
            maxfev=5000
        )
        
        lambda_0 = popt[0]
        fwhm = popt[1]
        q_factor = lambda_0 / fwhm if fwhm > 0 else None
        
        return q_factor, popt
    except:
        return None, None

def calculate_fsr(resonance_wavelengths):
    """Calculate Free Spectral Range from resonance positions."""
    if len(resonance_wavelengths) < 2:
        return None, None
    
    fsr_values = np.diff(resonance_wavelengths)
    mean_fsr = np.mean(np.abs(fsr_values))
    return mean_fsr, fsr_values

In [ ]:
# Find resonances
resonance_wls, resonance_depths, resonance_indices = find_resonances(
    wl_array, transmission_array, min_depth=0.03
)

print("=" * 60)
print("RING RESONATOR ANALYSIS RESULTS")
print("=" * 60)

# Report resonances
print(f"\nFound {len(resonance_wls)} resonance(s):")
for i, (wl, depth) in enumerate(zip(resonance_wls, resonance_depths)):
    print(f"  Resonance {i+1}: lambda = {wl*1e9:.2f} nm, Depth = {depth:.1%}")

# Calculate FSR
if len(resonance_wls) >= 2:
    mean_fsr, fsr_values = calculate_fsr(resonance_wls)
    print(f"\nFree Spectral Range (FSR):")
    print(f"  Measured FSR = {mean_fsr*1e9:.2f} nm")
    
    # Theoretical FSR for comparison
    n_g = N_CORE  # Group index approximation
    fsr_theory = CENTER_WL**2 / (2 * np.pi * RING_RADIUS * n_g)
    print(f"  Theoretical FSR ~ {fsr_theory*1e9:.2f} nm (approximate)")
else:
    print("\nNot enough resonances found to calculate FSR.")
    print("Consider expanding wavelength range.")

# Calculate Q-factors
print(f"\nQuality Factors:")
q_factors = []
fit_params_list = []
for i, res_idx in enumerate(resonance_indices):
    q_factor, fit_params = calculate_q_factor(wl_array, transmission_array, res_idx)
    q_factors.append(q_factor)
    fit_params_list.append(fit_params)
    if q_factor:
        print(f"  Resonance {i+1}: Q = {q_factor:.0f}")
    else:
        print(f"  Resonance {i+1}: Q-factor fitting failed (need more data points)")

In [ ]:
# Visualization with fitted curves
fig, ax = plt.subplots(figsize=(12, 6))

# Plot measured transmission
ax.plot(wl_array * 1e9, transmission_array, 'b-o', linewidth=2, markersize=8, 
        label='FDTD Simulation', zorder=3)

# Plot fitted Lorentzians
wl_fine = np.linspace(wl_array[0], wl_array[-1], 500)
for i, (res_idx, fit_params) in enumerate(zip(resonance_indices, fit_params_list)):
    if fit_params is not None:
        fitted_curve = lorentzian(wl_fine, *fit_params)
        q = q_factors[i]
        label = f'Lorentzian Fit (Q={q:.0f})' if q else 'Lorentzian Fit'
        ax.plot(wl_fine * 1e9, fitted_curve, '--', linewidth=2, 
                label=label, alpha=0.7)

# Mark resonances
if len(resonance_indices) > 0:
    ax.scatter(resonance_wls * 1e9, transmission_array[resonance_indices], 
               color='red', s=150, zorder=5, marker='v', label='Resonances')

ax.set_xlabel('Wavelength (nm)', fontsize=14)
ax.set_ylabel('Transmission', fontsize=14)
ax.set_title('Ring Resonator Transmission with Analysis', fontsize=16)
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.15)

plt.tight_layout()
plt.show()

## 8. 3D Verification Simulation

We run a quick 3D simulation at a single wavelength to verify proper mode propagation in the realistic waveguide structure with finite thickness.

In [ ]:
# 3D simulation parameters
WG_HEIGHT = 0.22 * um      # Waveguide height (typical Si3N4)
SUBSTRATE_DEPTH = 1 * um   # SiO2 substrate
TOTAL_DEPTH = 2.5 * um     # Total domain depth

print("3D Verification Parameters:")
print(f"  Waveguide height: {WG_HEIGHT*1e9:.0f} nm")
print(f"  Substrate depth: {SUBSTRATE_DEPTH*1e6:.1f} um")
print(f"  Total depth: {TOTAL_DEPTH*1e6:.1f} um")

In [ ]:
# Create 3D design
design_3d = Design(
    width=domain_width, 
    height=domain_height, 
    depth=TOTAL_DEPTH,
    material=Material(1.0)
)

# Add SiO2 substrate
design_3d += Rectangle(
    position=(0, 0, 0),
    width=domain_width,
    height=domain_height,
    depth=SUBSTRATE_DEPTH,
    material=Material(N_CLAD**2)
)

# Add waveguide structures from GDS with Si3N4 material
for layer_num, structures in gds_design.layers.items():
    for structure in structures:
        # Shift vertices and add z-coordinate
        shifted_vertices = [(v[0] + x_offset, v[1] + y_offset, SUBSTRATE_DEPTH) 
                          for v in structure.vertices]
        design_3d += Polygon(
            vertices=shifted_vertices, 
            material=Material(N_CORE**2),
            depth=WG_HEIGHT
        )

# Visualize 3D design
design_3d.show()

In [ ]:
# Calculate 3D grid parameters (coarser for memory)
WL_3D = CENTER_WL
DX_3D, DT_3D = calc_optimal_fdtd_params(
    WL_3D, N_CORE, 
    dims=3, 
    safety_factor=0.9, 
    points_per_wavelength=8,
    width=domain_width,
    height=domain_height,
    depth=TOTAL_DEPTH
)

# Shorter simulation time for verification
TIME_3D = 25 * WL_3D / LIGHT_SPEED
time_steps_3d = np.arange(0, TIME_3D, DT_3D)

print(f"3D Grid resolution: {DX_3D*1e9:.1f} nm")
print(f"Time steps: {len(time_steps_3d)}")
print(f"Simulation time: {TIME_3D*1e12:.2f} ps")

In [ ]:
# Rasterize 3D design
grid_3d = design_3d.rasterize(resolution=DX_3D)

# 3D Source signal
signal_3d = ramped_cosine(
    time_steps_3d, 
    amplitude=1.0, 
    frequency=LIGHT_SPEED/WL_3D,
    ramp_duration=WL_3D * 6 / LIGHT_SPEED, 
    t_max=TIME_3D / 2
)

# 3D Mode source at waveguide center height
# Use detected waveguide positions
wg_center_z = SUBSTRATE_DEPTH + WG_HEIGHT / 2
source_x_3d = WG_X_MIN + 2*WL_3D

source_3d = ModeSource(
    grid=grid_3d,
    center=(source_x_3d, BUS_INPUT_Y, wg_center_z),
    width=WG_WIDTH * 1.5,
    wavelength=WL_3D,
    pol="te",  # TE-like mode for thin waveguide
    signal=signal_3d,
    direction="+x"
)

# XY plane monitor at waveguide center height
monitor_xy = Monitor(
    start=(0, 0, wg_center_z),
    end=(domain_width, domain_height, wg_center_z),
    name="xy_plane"
)

print(f"Waveguide x-extent: [{WG_X_MIN*1e6:.2f}, {WG_X_MAX*1e6:.2f}] um")
print(f"3D Source position: ({source_x_3d*1e6:.2f}, {BUS_INPUT_Y*1e6:.2f}, {wg_center_z*1e6:.3f}) um")

In [ ]:
# Create and run 3D simulation
sim_3d = Simulation(
    design=design_3d,
    devices=[source_3d, monitor_xy],
    boundaries=[PML(edges='all', thickness=0.75*WL_3D)],
    time=time_steps_3d,
    resolution=DX_3D
)

# Run with live animation
sim_3d.run(
    animate_live="Ez",
    animation_interval=5,
    clean_visualization=True,
    cmap="twilight_zero"
)

In [ ]:
# Plot 3D simulation results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Final field distribution
if monitor_xy.fields['Ez']:
    final_field = monitor_xy.fields['Ez'][-1]
    im = axes[0].imshow(final_field, cmap='RdBu', origin='lower', aspect='auto')
    plt.colorbar(im, ax=axes[0], label='Ez amplitude')
    axes[0].set_xlabel('X index')
    axes[0].set_ylabel('Y index')
    axes[0].set_title('Ez Field Distribution (3D Simulation)')
else:
    axes[0].text(0.5, 0.5, 'No field data recorded', ha='center', va='center')
    axes[0].set_title('Ez Field Distribution')

# Power evolution
if monitor_xy.power_history:
    axes[1].plot(monitor_xy.power_history, 'r-', linewidth=2)
    axes[1].set_xlabel('Time step')
    axes[1].set_ylabel('Power')
    axes[1].set_title('Power Evolution in 3D Simulation')
    axes[1].grid(True, alpha=0.3)
else:
    axes[1].text(0.5, 0.5, 'No power data recorded', ha='center', va='center')
    axes[1].set_title('Power Evolution')

plt.tight_layout()
plt.show()

## 9. Summary

This notebook demonstrated a complete workflow for ring resonator simulation:

### Design Parameters
| Parameter | Value |
|-----------|-------|
| Ring radius | 6 um |
| Waveguide width | 500 nm |
| Coupling gap | 200 nm |
| Core material | Si3N4 (n=2.04) |
| Cladding material | SiO2 (n=1.444) |
| Center wavelength | 1550 nm |

### Key Results
- Created ring resonator geometry using gdsfactory
- Imported GDS into beamz for FDTD simulation
- Extracted transmission spectrum through wavelength sweep
- Calculated resonance parameters (Q-factor, FSR)
- Verified mode propagation with 3D simulation

### Next Steps
- Optimize coupling gap for desired coupling coefficient
- Explore add-drop ring resonator configuration
- Investigate thermal tuning effects
- Design cascaded ring filters for sharper roll-off

In [ ]:
# Clean up temporary GDS file
import os
if os.path.exists(gds_path):
    os.remove(gds_path)
    print(f"Cleaned up temporary file: {gds_path}")

print("\nNotebook complete!")